# 实验-1-介绍 Phi-3.5 指令


### 使用optimum-cli将Phi-3-mini-4k-instruct模型转换为INT4


In [ ]:
from pathlib import Path

llm_model_id = "microsoft/Phi-3.5-mini-instruct"
llm_model_path = "../model/phi-3.5-mini-instruct-ov"

if not Path(llm_model_path).exists():
    !optimum-cli export openvino --model {llm_model_id} --task text-generation-with-past --weight-format int4 --group-size 128 --ratio 0.6  --sym  --trust-remote-code {llm_model_path}

In [ ]:
from transformers import AutoConfig, AutoTokenizer
from optimum.intel.openvino import OVModelForCausalLM

In [ ]:
ov_config = {"PERFORMANCE_HINT": "LATENCY", "NUM_STREAMS": "1", "CACHE_DIR": ""}

In [ ]:
ov_model = OVModelForCausalLM.from_pretrained(
    llm_model_path,
    device='GPU',
    ov_config=ov_config,
    config=AutoConfig.from_pretrained(llm_model_path, trust_remote_code=True),
    trust_remote_code=True,
)

In [ ]:
tok = AutoTokenizer.from_pretrained(llm_model_path, trust_remote_code=True)

In [ ]:
tokenizer_kwargs =  {"add_special_tokens": False}

In [ ]:
prompt = "<|user|>\n你了解 .NET 吗?\n<|end|><|assistant|>\n"
input_tokens = tok(prompt, return_tensors="pt", **tokenizer_kwargs)

In [ ]:
answer = ov_model.generate(**input_tokens, max_new_tokens=1024)

In [ ]:
tok.batch_decode(answer, skip_special_tokens=True)[0]


---

**免责声明**：  
本文档使用AI翻译服务[Co-op Translator](https://github.com/Azure/co-op-translator)进行翻译。尽管我们努力确保翻译的准确性，但请注意，自动翻译可能包含错误或不准确之处。原始语言的文档应被视为权威来源。对于关键信息，建议使用专业人工翻译。我们不对因使用此翻译而产生的任何误解或误读承担责任。
